<a target="_blank" href="https://colab.research.google.com/github/Jasonchen0317/CSGY-6613-Assignment/blob/main/assignment-2/assignment_2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from sklearn.preprocessing import scale
import nltk 
from nltk.corpus import stopwords
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import optuna

In [2]:
def word_extraction(sentence):    
    ignore = ['a', "the", "is"]   
    words = re.sub("[^\w]", " ",  sentence).split()    
    cleaned_text = [w.lower() for w in words if w not in ignore]    
    return cleaned_text

In [3]:
def tokenize(sentences):    
    words = []    
    for sentence in sentences:              
        words.extend(sentence)           
    words = sorted(list(set(words)))    
    return words

In [4]:
def generate_bow(allsentences, v):     
    print("Word List for Document \n{0} \n".format(v))
    bow = []
    n=0
    for sentence in allsentences:
        n=n+1
        print('sentence: '+str(n))            
        bag_vector = [0] * len(v)        
        for w in sentence:
            if(w==2):
                continue            
            for i,word in enumerate(v):                
                if word == w:                     
                    bag_vector[i] += 1                            
       # print("{0}\n{1}\n".format(sentence,np.array(bag_vector)))
        bow.append(bag_vector)
    return bow

In [74]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    path='imdb.npz',
    num_words=10000,
    skip_top=100
)

print(x_train)
print(y_train)

[list([2, 2, 2, 2, 2, 530, 973, 1622, 1385, 2, 458, 4468, 2, 3941, 2, 173, 2, 256, 2, 2, 100, 2, 838, 112, 2, 670, 2, 2, 2, 480, 284, 2, 150, 2, 172, 112, 167, 2, 336, 385, 2, 2, 172, 4536, 1111, 2, 546, 2, 2, 447, 2, 192, 2, 2, 2, 147, 2025, 2, 2, 2, 2, 1920, 4613, 469, 2, 2, 2, 2, 2, 2, 2, 530, 2, 2, 2, 2, 1247, 2, 2, 2, 515, 2, 2, 2, 626, 2, 2, 2, 2, 386, 2, 2, 316, 2, 106, 2, 2, 2223, 5244, 2, 480, 2, 3785, 2, 2, 130, 2, 2, 2, 619, 2, 2, 124, 2, 2, 135, 2, 2, 1415, 2, 2, 2, 2, 215, 2, 2, 2, 2, 2, 407, 2, 2, 2, 2, 2, 107, 117, 5952, 2, 256, 2, 2, 2, 3766, 2, 723, 2, 2, 2, 530, 476, 2, 400, 317, 2, 2, 2, 2, 1029, 2, 104, 2, 2, 381, 2, 297, 2, 2, 2071, 2, 2, 141, 2, 194, 7486, 2, 2, 226, 2, 2, 134, 476, 2, 480, 2, 144, 2, 5535, 2, 2, 2, 2, 224, 2, 2, 104, 2, 226, 2, 2, 2, 1334, 2, 2, 2, 283, 2, 2, 4472, 113, 103, 2, 2, 2, 5345, 2, 178, 2])
 list([2, 194, 1153, 194, 8255, 2, 228, 2, 2, 1463, 4369, 5012, 134, 2, 2, 715, 2, 118, 1634, 2, 394, 2, 2, 119, 954, 189, 102, 2, 207, 110, 3103, 

In [75]:
for x in x_train:
    x[:] = [value for value in x if value != 2]
for x in x_test:
    x[:] = [value for value in x if value != 2]

In [76]:
vocab_train = tokenize(x_train)
vocab_test = tokenize(x_test)
vocab = list(set(vocab_test) | set(vocab_train))
print(len(vocab))

9900


In [77]:
bow_train = np.array(generate_bow(x_train[:5000], vocab))
bow_test = np.array(generate_bow(x_test[:500], vocab))

Word List for Document 
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 

In [78]:
def create_optimizer(trial):
    # We optimize the choice of optimizers as well as their parameters.
    kwargs = {}
    optimizer_options = ["RMSprop", "Adam", "SGD"]
    optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    if optimizer_selected == "RMSprop":
        kwargs["learning_rate"] = trial.suggest_float(
            "rmsprop_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["weight_decay"] = trial.suggest_float("rmsprop_weight_decay", 0.85, 0.99)
        kwargs["momentum"] = trial.suggest_float("rmsprop_momentum", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "Adam":
        kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
    elif optimizer_selected == "SGD":
        kwargs["learning_rate"] = trial.suggest_float(
            "sgd_opt_learning_rate", 1e-5, 1e-1, log=True
        )
        kwargs["momentum"] = trial.suggest_float("sgd_opt_momentum", 1e-5, 1e-1, log=True)

    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer

In [84]:
def objective_logistic(trial):
    
    # Build model and optimizer.
    model = Sequential()
    model.add(Dropout(0.5 , input_dim=bow_train.shape[1]))
    model.add(Dense(1,  activation='sigmoid')) 
    
    optimizer = create_optimizer(trial)

    model.compile(
        loss="binary_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"],
    )
    # Training and validating cycle.
    model.fit(
        bow_train,
        y_train[:5000],
        validation_data=(bow_test, y_test[:500]),
        shuffle=True,
        batch_size=trial.suggest_categorical("batch_size", [16, 32, 64, 128]),
        epochs=20,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(bow_test, y_test[:500], verbose=0)
    return score[1]



In [85]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_logistic, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-10-05 19:51:07,224] A new study created in memory with name: no-name-ed06cb02-244f-4510-b039-e5a71bfd2aee
[I 2023-10-05 19:51:25,298] Trial 0 finished with value: 0.7419999837875366 and parameters: {'optimizer': 'RMSprop', 'rmsprop_learning_rate': 0.04572833591504045, 'rmsprop_weight_decay': 0.9433195555211124, 'rmsprop_momentum': 0.00023472004878526213, 'batch_size': 32}. Best is trial 0 with value: 0.7419999837875366.
[I 2023-10-05 19:51:49,860] Trial 1 finished with value: 0.5879999995231628 and parameters: {'optimizer': 'SGD', 'sgd_opt_learning_rate': 4.096286104313986e-05, 'sgd_opt_momentum': 6.940778432802693e-05, 'batch_size': 16}. Best is trial 0 with value: 0.7419999837875366.
[I 2023-10-05 19:52:08,496] Trial 2 finished with value: 0.7419999837875366 and parameters: {'optimizer': 'RMSprop', 'rmsprop_learning_rate': 0.05450160702706831, 'rmsprop_weight_decay': 0.9050740785147249, 'rmsprop_momentum': 0.05359539248584222, 'batch_size': 32}. Best is trial 0 with value: 0.

KeyboardInterrupt: 

In [ ]:
adamhistory = 

In [66]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train)
print(y_train)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
[5 0 4 ... 5 6 8]


In [69]:
def objective_logistic_mnist(trial):
    # Get MNIST data.

    # Build model and optimizer.
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(10, input_dim=28*28, activation='softmax')) 
    
    optimizer = create_optimizer(trial)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"],
    )
    model.fit(
        x_train,
        y_train,
        validation_data=(x_test, y_test),
        shuffle=True,
        batch_size=trial.suggest_categorical("batch_size", [32, 64, 128]),
        epochs=trial.suggest_int("epochs", 20, 50),
        verbose=False,
    )
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

In [70]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_logistic_mnist, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-10-05 17:32:09,415] A new study created in memory with name: no-name-24d0f5fc-e90a-4ba5-aa67-74e4a98741ea
[I 2023-10-05 17:33:37,883] Trial 0 finished with value: 0.8888999819755554 and parameters: {'optimizer': 'SGD', 'sgd_opt_learning_rate': 0.07686860336326559, 'sgd_opt_momentum': 1.3025970669894536e-05, 'batch_size': 32, 'epochs': 34}. Best is trial 0 with value: 0.8888999819755554.
[I 2023-10-05 17:34:44,821] Trial 1 finished with value: 0.9139999747276306 and parameters: {'optimizer': 'RMSprop', 'rmsprop_learning_rate': 6.291824230287061e-05, 'rmsprop_weight_decay': 0.8860815455185826, 'rmsprop_momentum': 0.00010054405888113158, 'batch_size': 32, 'epochs': 26}. Best is trial 1 with value: 0.9139999747276306.
[I 2023-10-05 17:35:13,956] Trial 2 finished with value: 0.6039000153541565 and parameters: {'optimizer': 'RMSprop', 'rmsprop_learning_rate': 0.046924448170789085, 'rmsprop_weight_decay': 0.9822486640927084, 'rmsprop_momentum': 0.00026736887063055526, 'batch_size': 12

Number of finished trials: 20
Best trial:
  Value: 0.9139999747276306
  Params: 
    optimizer: RMSprop
    rmsprop_learning_rate: 6.291824230287061e-05
    rmsprop_weight_decay: 0.8860815455185826
    rmsprop_momentum: 0.00010054405888113158
    batch_size: 32
    epochs: 26


In [37]:
def create_model(trial):
    # We optimize the numbers of layers, their units and weight decay parameter.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    weight_decay = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)
        model.add(
            tf.keras.layers.Dense(
                num_hidden,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            )
        )
    model.add(
        tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
    )
    return model

In [38]:
def objective_multi(trial):
    # Get MNIST data.

    # Build model and optimizer.
    model = create_model(trial)
    
    optimizer = create_optimizer(trial)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"],
    )
    model.fit(
        x_train,
        y_train,
        validation_data=(x_test, y_test),
        shuffle=True,
        batch_size=128,
        epochs=20,
        verbose=False,
    )
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

In [39]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_multi, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-10-05 15:30:00,485] A new study created in memory with name: no-name-dfaef60b-f8e0-4f6b-b134-68ba44d9aaa3
[I 2023-10-05 15:30:26,176] Trial 0 finished with value: 0.10320000350475311 and parameters: {'n_layers': 3, 'weight_decay': 3.8640606535118497e-10, 'n_units_l0': 33, 'n_units_l1': 9, 'n_units_l2': 11, 'optimizer': 'RMSprop', 'rmsprop_learning_rate': 0.09740913043486431, 'rmsprop_weight_decay': 0.8712862366577435, 'rmsprop_momentum': 0.007518102227967097}. Best is trial 0 with value: 0.10320000350475311.
[I 2023-10-05 15:30:54,255] Trial 1 finished with value: 0.0957999974489212 and parameters: {'n_layers': 2, 'weight_decay': 1.762560536399303e-08, 'n_units_l0': 63, 'n_units_l1': 5, 'optimizer': 'SGD', 'sgd_opt_learning_rate': 0.0026575792393410467, 'sgd_opt_momentum': 0.0007489070244473785}. Best is trial 0 with value: 0.10320000350475311.
[I 2023-10-05 15:31:15,768] Trial 2 finished with value: 0.1136000007390976 and parameters: {'n_layers': 3, 'weight_decay': 0.000380620

Number of finished trials: 20
Best trial:
  Value: 0.5336999893188477
  Params: 
    n_layers: 2
    weight_decay: 2.3686266988407362e-08
    n_units_l0: 18
    n_units_l1: 120
    optimizer: RMSprop
    rmsprop_learning_rate: 2.125566620851011e-05
    rmsprop_weight_decay: 0.9893460807358169
    rmsprop_momentum: 0.07810128079359098


In [53]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [64]:
def create_cnn_model(trial):
    # We optimize the numbers of layers, their units and weight decay parameter.
    n_cov_layers = trial.suggest_int("n_cov_layers", 1, 3)
    n_layers = trial.suggest_int("n_layers", 1, 3)
    weight_decay = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    num_hidden = trial.suggest_int("num_hidden", 4, 128, log=True)
    
    model = tf.keras.Sequential()
    model.add(keras.Input(shape=(32,32,3)))
    
    for i in range(n_cov_layers):
        model.add(
            layers.Conv2D(
                filters=trial.suggest_categorical("filters", [32, 64]),
                kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
                strides=trial.suggest_categorical("strides", [1, 2]),
                activation=trial.suggest_categorical("activation", ["relu", "linear"]),
            )
            
        )
        model.add(
            layers.MaxPooling2D(
                pool_size=trial.suggest_categorical("pool_size", [2, 3]),
                strides=trial.suggest_categorical("strides", [1, 2]),
            )
        )
    model.add(Flatten())
    model.add(
        tf.keras.layers.Dense(
            num_hidden,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
        )
    )
    model.add(
        tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
    )
    return model

In [62]:
def objective_cnn(trial):
    # Get MNIST data.

    # Build model and optimizer.
    model = create_cnn_model(trial)
    
    optimizer = create_optimizer(trial)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"],
    )
    model.fit(
        x_train,
        y_train,
        validation_data=(x_test, y_test),
        shuffle=True,
        batch_size=128,
        epochs=20,
        verbose=False,
    )
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

In [65]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_cnn, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-10-05 16:06:13,944] A new study created in memory with name: no-name-d4340bc7-4bd6-4ae2-9d32-b93a829741fd
[I 2023-10-05 16:13:19,435] Trial 0 finished with value: 0.10000000149011612 and parameters: {'n_cov_layers': 1, 'n_layers': 2, 'weight_decay': 2.1440442210415558e-08, 'num_hidden': 57, 'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'relu', 'pool_size': 2, 'optimizer': 'SGD', 'sgd_opt_learning_rate': 0.009971830398396862, 'sgd_opt_momentum': 0.00015527099605150811}. Best is trial 0 with value: 0.10000000149011612.
[I 2023-10-05 16:16:21,661] Trial 1 finished with value: 0.10000000149011612 and parameters: {'n_cov_layers': 2, 'n_layers': 2, 'weight_decay': 0.0004237218196259068, 'num_hidden': 8, 'filters': 64, 'kernel_size': 3, 'strides': 2, 'activation': 'linear', 'pool_size': 3, 'optimizer': 'Adam', 'adam_learning_rate': 0.0010049573636158702}. Best is trial 0 with value: 0.10000000149011612.
[I 2023-10-05 16:25:34,364] Trial 2 finished with value: 0.10000000

KeyboardInterrupt: 